In [21]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity

# 실행

In [84]:
find_sim_movie_ver1(products_df,content_sim(search_df(products_df)),7).sort_values(by=["ch_score","weighted_vote"], ascending=False).iloc[:,[1,2,3,23,24,22]]

검색어를 입력하시오: Romantic 러그
중요한 특성의 순위를 매기시오: 디자인 가격 배송 내구성


,product_name,product_price,Product_Url,contents,ch_score,weighted_vote
544,빈티지그린 로즈부케 드로잉 원형러그,83900,https://ohou.se/productions/517743/selling,"[Romantic, 카페트, 러그, 러그]",4.750000,4.608768
693,심플 원형 러그 사계절 단모 극세사 워셔블 카페트 4colors 3size,17900,https://ohou.se/productions/496158/selling,"[Romantic, 카페트, 러그, 러그]",4.697279,4.670166
871,로코코 플로랄 드로잉 원형러그,65900,https://ohou.se/productions/229792/selling,"[Romantic, 카페트, 러그, 러그]",4.692179,4.628153
614,터치미 극세사 거실 러그 사계절 먼지없는 단모 카페트 4size 4colors,16800,https://ohou.se/productions/489858/selling,"[Romantic, 카페트, 러그, 러그]",4.580000,4.590708
857,파렛트 모던 디자인 극세사 원형 러그 2size,34900,https://ohou.se/productions/279033/selling,"[Romantic, 카페트, 러그, 러그]",4.567857,4.575663


# 1. 데이터셋

## 1.1 데이터셋 불러오기

In [49]:
products_df = pd.read_excel("products_df.xlsx")

In [50]:
products_df["contents"] = products_df['contents'].apply(literal_eval)

## 1.2 검색어

In [82]:
def search_df(df):
    dic = dict()
    search = input("검색어를 입력하시오: ").split()
    character = input("중요한 특성의 순위를 매기시오: ").split()
    
    #특성별 가중치
    for x in range(0,len(character)):
        dic[x] =character[x]
    ch = pd.concat([df[dic[0]]*0.4,df[dic[1]]*0.3,df[dic[2]]*0.2,df[dic[3]]*0.1],axis=1)
    ch["ch_score"] = ch["내구성"]+ch["가격"]+ch["디자인"]+ch["배송"]
    df["ch_score"] = ch["ch_score"].values
    
    df.loc[len(df)] = ["검색어","0","0","0","0",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,search,0]
    df['contents_literal'] = df['contents'].apply(lambda x : (' ').join(x))
    df["contents_len"] = df["contents"].apply(lambda x: len(x))
    
    return df

# 2. CBF

## 2.1 콘텐츠 유사도 측정

In [75]:
def content_sim(df):
    count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2)) 
    content_mat = count_vect.fit_transform(df['contents_literal'])
    content_sim = cosine_similarity(content_mat, content_mat)
    content_sim_sorted_ind = content_sim.argsort()[:, ::-1]
    
    return content_sim_sorted_ind

## 2.2 CBF 추천 알고리즘

In [83]:
def find_sim_movie_ver1(df, sorted_ind, top_n):
    title_index = len(df)-1
    similar_indexes = sorted_ind[title_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes][df.iloc[similar_indexes]["contents_len"]>1]